In order to make it work, you’ll need to either click “**Run All**” or run each cell one by one from the beginning.

This code loads structured data from a URL and converts it into a Pandas DataFrame.
It supports two formats: JSON and XML (including .xml.gz files compressed with GZIP).

**1. Format detection**

  - If the HTTP response header contains Content-Type: application/json, or if the URL ends with .json or contains "rest-api", the content is treated as JSON.

  - Otherwise, the content is treated as XML.

**2. JSON processing**

 - Can handle:

    + A list of objects ([ {...}, {...} ])

    + A dictionary containing a "jobs" key ({"jobs": [ {...}, {...} ]})

    + Any other JSON object that can be converted into a DataFrame.

**3. XML processing**

- Works with both standard .xml files and .xml.gz (GZIP compressed).

- Searches for elements matching the job_tag parameter (default: "job") and extracts their child tags and values.

4. Data cleaning **bold text**

- Truncates overly long text values.

- Replaces invalid numbers (NaN or infinite values) with NaN.

- Removes columns that are entirely empty, or fills missing values with 0.

**In short, load_feed_to_dataframe() automatically detects the data format, reads it, cleans it, and returns it as a ready-to-use Pandas DataFrame for analysis.**


In [ ]:
# @title
##JSON FUNCTION

import pandas as pd
import requests
import xml.etree.ElementTree as ET
import numpy as np
from io import BytesIO
import gzip
import datetime

# --- Aux functions ---

def truncate(value, max_length=49000):
    if value and isinstance(value, str) and len(value) > max_length:
        return value[:max_length]
    return value

def clean_invalid_numbers(df):
    return df.apply(lambda col: col.map(lambda x: np.nan if isinstance(x, float) and (np.isnan(x) or np.isinf(x)) else x))


def import_datetime(df):
    df['last_update'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def drop_all_nulls(df):
    for column in df.columns:
        if df[column].isnull().all():
            df.drop(columns=[column], inplace=True)
            print(f"Column '{column}' has been dropped due to all NaN values.")
        else:
            df[column].fillna(0, inplace=True)

# --- PANDAS. Main function to load the XML ---

def load_feed_to_dataframe(url, job_tag="job"):
    """
    Loads an XML feed (.xml or .xml.gz) or JSON from a URL and converts it into a DataFrame.

    Args:
        url (str): URL of the feed.
        job_tag (str): Name of the XML tag representing each job (only for XML feeds).

    Returns:
        pd.DataFrame: DataFrame containing the feed data.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()

        # Try to parse as JSON if the content-type indicates it or the URL suggests .json
        if "application/json" in response.headers.get("Content-Type", "") or url.endswith(".json") or "rest-api" in url:
            data = response.json()

            # Handle different JSON formats
            if isinstance(data, list):
                df = pd.DataFrame(data)
            elif isinstance(data, dict) and "jobs" in data:
                df = pd.DataFrame(data["jobs"])
            else:
                df = pd.DataFrame(data)

            # Truncate and clean
            df = df.applymap(lambda x: truncate(x) if isinstance(x, str) else x)
            df = clean_invalid_numbers(df)
            return df

        # If not JSON, treat as XML
        if url.endswith(".gz"):
            with gzip.GzipFile(fileobj=BytesIO(response.content)) as f:
                xml_content = f.read()
        else:
            xml_content = response.content

        root = ET.fromstring(xml_content)
        items = root.findall(f".//{job_tag}")
        if not items:
            print(f"No <{job_tag}> elements found in the XML.")
            return pd.DataFrame()

        jobs_data = []
        for job in items:
            job_data = {child.tag: truncate(child.text) for child in job}
            jobs_data.append(job_data)

        df = pd.DataFrame(jobs_data)
        df = clean_invalid_numbers(df)
        return df

    except Exception as e:
        print(f"Error processing the feed: {e}")
        return pd.DataFrame()



In [ ]:
# =====================================================================================
feed_url = input("Feed URL?  ")

df = load_feed_to_dataframe(feed_url)


# =====================================================================================
#Change NaN with 0
print(f"NUMBER OF ROWS: {df.shape[0]}")
print(f"NUMBER OF COLUMNS: {df.shape[1]}")
print("NAN VALUES:")
print(df.isnull().sum())

#Transform all the Nan values into 0.
df = df.fillna(0)

'''
for column in df.columns:
    if df[column].isnull().all():
        df = df.drop(columns=[column])
        print(f"Column '{column}' has been dropped due to all NaN values.")
    else:
      print(f"Column '{column}' has values.")
    '''

# =====================================================================================
df.head() #it shows the df with the first 5 rows


In [ ]:
df.info()

In [ ]:
# =====================================================================================
# Run this cell to download a .csv file with feed

df.to_csv('feed.csv', index=False)
from google.colab import files
files.download('feed.csv')


In [ ]:
# =====================================================================================
# Write the name of the CLIENT in the opt2 that you want to downlowad (ie:Enhance). It must have this structure: 'xxxxx'

opt2 = 'FOruntosd'

# Keep in mind that you will be reading all the rows that match the value in opt2 in the column client (df.client). If you want to read another column, just change client for name of the columna that you want to filter
company = df[df.clientname.str.contains(opt2, na=False)]
company.head()

# Run this cell to download a .csv file with the CLIENT given

company.to_csv(f'{opt2}.csv', index=False)
from google.colab import files
files.download(f'{opt2}.csv')
